## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.1"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.1

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_1hrgcnQidng7cWb4a99HHWsk


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')
today_date
```

I called the tool named crawl with these arguments:
```
{"q":"things to do in Atlanta on 2024-10-03","textFormat":"HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-10-03"},"webPages":{"value":[{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... This guide is your gateway to discovering the best things to do in Atlanta, ... 03. 2024. 7:00 PM Thu. La Luz. Purgatory Stage at Masquerade - GA | Capacity: 300. 30303, 75 MLK Jr Dr SW, Atlanta, GA, US"},{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Events From October 1 To The Weekend Of October 5-6 2024. \u2192 See September events. \u2192 For concerts this week, see Atlanta concerts in October 2024. \u2013 Daddy Long Legs, September 13 - October 6 2024. Musical at The Legacy Theatre, Tyrone. More details."},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids - MSN","url":"https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","displayUrl":"https://www.msn.com/.../33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","snippet":"Oktoberfest in Helen Helen. Through Sunday, October 27, 2024. Head north about 1.5 hours to join the German celebration in Helen for its 53rd year."},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898","snippet":"3. World of Coca-Cola. 14,732. Speciality Museums. Admission tickets from $23. Immerse yourself in World of Coca-Cola, the one and only place you can explore the story of the world\u0026#39;s most popular beverage brand. It\u0026#39;s a place where anyone can experience moments of happiness and\u2026. See ways to experience (16) 2024.","thumbnailUrl":"https://www.bing.com/th?id=OIP.1g_oKFZuxZyu0MwSWAr2YwHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Stay up to date on things to do and events in Atlanta today, this week and this weekend. Whether you\u2019re into live music, food and wine, sports or family-friendly entertainment, Atlanta events offer nonstop fun for everyone. ... Multiple days until 11/03/2024. Atlanta Art Week 10/2/24 \u2013 10/6/24. Games Week Georgia 10/2/24 \u2013 10/7/24 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","snippet":"2024. 2. Atlanta Botanical Garden. 5,236. Gardens. Step into a world of magic and serenity at the Atlanta Botanical Garden, where features like the Fuqua Orchid Center treat visitors to a rare collection of high-elevation orchids never before grown\u2026. See tours. 3. World of Coca-Cola."},{"name":"The best things to do in Atlanta right now - Time Out","url":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta","displayUrl":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta","snippet":"4. Martin Luther King Jr. National Historical Park. The Martin Luther King Centre is the must-see tourist attraction in Atlanta. Once \u2018The King Center,\u2019 it is now a nonprofit organization in ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.-XmrGhWhWtlBL3xZRZjcJwHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover Atlanta\u2019s Anticipated 2024 Highlights","url":"https://discoveratlanta.com/stories/things-to-do/discover-atlantas-anticipated-2024-highlights/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/discover-atlantas-anticipated-2024...","snippet":"Signia by Hilton Atlanta, the city\u2019s largest Downtown, ground-up hotel development in 40 years, opens in January 2024. The hotel\u2019s eight food and beverages offerings center around Georgia\u2019s Southern history. The 976-room hotel will introduce a three full-service restaurants and three bars. Highlights include Capolinea, an Italian-inspired ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.3Ek-SE8inqMg2SLHb7iNDgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.GDtNm5tAuiy9g1VcZR9skwHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/07/24-Things-to-Do-in-Downtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/things-to-do-in-downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.pTFzMm6LiPCR7Yr9RrSvhwAAAA\u0026pid=Api","contentUrl":"https://tourscanner.com/blog/wp-content/uploads/2020/11/fun-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://tourscanner.com/blog/fun-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cGQLTADJ7EIEL8n56X81XwHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TNv2qzXsSFvw43t4zu0gGgHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/3NRm4q9Qg2bdUqXvenPBVeIloV4=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.gpxEMgIaDnDSvyMtPJjjdgHaE6\u0026pid=Api","contentUrl":"https://wowtravel.me/wp-content/uploads/2022/06/Top-10-Things-to-Do-in-Atlanta.jpg","hostPageUrl":"https://wowtravel.me/things-to-do-in-atlanta-this-week/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6_fMiPMrfeqD9K0vgkdR6QHaE8\u0026pid=Api","contentUrl":"https://www.getawaymavens.com/wp-content/uploads/2019/01/Westin-Peachtree-Atlanta.jpg","hostPageUrl":"https://www.getawaymavens.com/downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.vjZ_V7Yjzfqyphfsj_SBjgAAAA\u0026pid=Api","contentUrl":"https://daytripqueen.com/wp-content/uploads/2021/07/Free-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://daytripqueen.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/Vl6GPUuj1mc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=Vl6GPUuj1mc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Best-Things-to-Do-in-Atlanta-in-2023.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.k7hGzw27Lx3uqX8C1eseEQHaE6\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2022/05/atlanta-1280x850.jpeg","hostPageUrl":"https://365atlantatraveler.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.9mBzTpymEhPfPfnOyuyhzAAAAA\u0026pid=Api","contentUrl":"https://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Yac324FtG-dH_KY8XaUTZQHaME\u0026pid=Api","contentUrl":"https://www.frostandsun.com/wp-content/uploads/2017/11/things-to-do-in-atlanta-pin-me-1-628x1024.jpg.webp","hostPageUrl":"https://www.frostandsun.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zsUlule-ACLeXXDRofPdiAAAAA\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"}]}}
```

Here are some events and activities happening in Atlanta today, October 3, 2024:

1. **La Luz Concert**:
   - Venue: Purgatory Stage at Masquerade
   - Time: 7:00 PM
   - [More Info](https://atlanta-ga.events/october/)

2. **Atlanta Art Week**:
   - Dates: October 2-6, 2024
   - [More Info](https://discoveratlanta.com/events/main/)

3. **Games Week Georgia**:
   - Dates: October 2-7, 2024
   - [More Info](https://discoveratlanta.com/events/main/)

4. **Visit the World of Coca-Cola**:
   - A popular attraction in Atlanta where you can explore the history and enjoy tastings of Coca-Cola.
   - [More Info](https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html)

5. **Martin Luther King Jr. National Historical Park**:
   - A must-see tourist attraction in Atlanta.
   - [More Info](https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta)

6. **Atlanta Botanical Garden**:
   - A serene place to visit with features like the Fuqua Orchid Center.
   - [More Info](https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html)

These are some of the events and activities you can enjoy in Atlanta today!


In [4]:
output.LastMessage.DisplayAs("text/markdown");

Here are some events and activities happening in Atlanta today, October 3, 2024:

1. **La Luz Concert**:
   - Venue: Purgatory Stage at Masquerade
   - Time: 7:00 PM
   - [More Info](https://atlanta-ga.events/october/)

2. **Atlanta Art Week**:
   - Dates: October 2-6, 2024
   - [More Info](https://discoveratlanta.com/events/main/)

3. **Games Week Georgia**:
   - Dates: October 2-7, 2024
   - [More Info](https://discoveratlanta.com/events/main/)

4. **Visit the World of Coca-Cola**:
   - A popular attraction in Atlanta where you can explore the history and enjoy tastings of Coca-Cola.
   - [More Info](https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html)

5. **Martin Luther King Jr. National Historical Park**:
   - A must-see tourist attraction in Atlanta.
   - [More Info](https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta)

6. **Atlanta Botanical Garden**:
   - A serene place to visit with features like the Fuqua Orchid Center.
   - [More Info](https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html)

These are some of the events and activities you can enjoy in Atlanta today!

In [5]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Get the dates for the next four weekends
weekend_dates = []
for i in range(4):
    # Calculate the coming Saturday
    saturday = today + timedelta(days=(5 - today.weekday()) % 7 + 7 * i)
    weekend_dates.append(saturday.strftime('%Y-%m-%d'))

weekend_dates
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on 2024-10-05", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-10-05"},"webPages":{"value":[{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4898908883932841\u0026mkt=en-US\u0026setlang=en-US\u0026w=t-7sQmKPkPazxHogtJy0oBuKycEPAm_-"},{"name":"Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Venture Atlanta 2024 10/8/24 \u2013 10/9/24. Atlanta miniFEST 10/9/24 \u2013 10/10/24. Geology Walk and Talk 10/9/24. ... October 05. Saturday. Big Green Egg \u201CEGGtoberfest\u201D: Celebrating 50 Years 10/5/24 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4819881477614541\u0026mkt=en-US\u0026setlang=en-US\u0026w=cQIKbv56z1uP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids - MSN","url":"https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","displayUrl":"https://www.msn.com/.../33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","snippet":"Oktoberfest in Helen Helen. Through Sunday, October 27, 2024. Head north about 1.5 hours to join the German celebration in Helen for its 53rd year."},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4714289709671445\u0026mkt=en-US\u0026setlang=en-US\u0026w=uP6_4hpqL9Zfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"10 things to do in Atlanta this weekend - accessatlanta.com","url":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this-weekend/EIBV4IR6EJD6XHWXAQRYTDQBVU/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this...","snippet":"Best Things to do in Atlanta this weekend. DreamHack Atlanta is the ultimate gaming and esports festival, where gamers, streamers, and fans unite for an epic weekend of non-stop action!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=871016790476\u0026mkt=en-US\u0026setlang=en-US\u0026w=XM5LK6so-BR50MJBphof8Eu0vDoQC3dg"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","snippet":"2024. 2. Atlanta Botanical Garden. 5,236. Gardens. Step into a world of magic and serenity at the Atlanta Botanical Garden, where features like the Fuqua Orchid Center treat visitors to a rare collection of high-elevation orchids never before grown\u2026. See tours. 3. World of Coca-Cola.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4999832017579678\u0026mkt=en-US\u0026setlang=en-US\u0026w=aJsbstGA5Y_hqWtUQWQStMnnppmx6J8T"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4885792048087312\u0026mkt=en-US\u0026setlang=en-US\u0026w=UWMF4npb9HKtzRRUG028yHS95G2gOIKx"},{"name":"The best things to do in Atlanta right now - Time Out","url":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta","displayUrl":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta","snippet":"4. Martin Luther King Jr. National Historical Park. The Martin Luther King Centre is the must-see tourist attraction in Atlanta. Once \u2018The King Center,\u2019 it is now a nonprofit organization in ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4565877110487686\u0026mkt=en-US\u0026setlang=en-US\u0026w=FYuFUogV2_4cUlw5_DIQpKVX3mnuh8Iq","thumbnailUrl":"https://www.bing.com/th?id=OIP.-XmrGhWhWtlBL3xZRZjcJwHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The 17 Best Things to Do in Atlanta - Cond\u00E9 Nast Traveler","url":"https://www.cntraveler.com/gallery/best-things-to-do-in-atlanta","displayUrl":"https://www.cntraveler.com/gallery/best-things-to-do-in-atlanta","snippet":"Atlanta Botanical Garden. Adjacent to Atlanta\u2019s verdant Piedmont Park, it\u2019s easy to see why this 30-acre oasis in the heart of Midtown welcomes more than half a million visitors each year. In ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.k6p7QyZP1cNKZi0qNguohAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898","snippet":"1. Georgia Aquarium. 23,867. Aquariums. Georgia Aquarium contains more than 70 amazing habitats, with thousands of marine animals such as whale sharks, manta rays, beluga whales, and more. Don\u2019t miss out on the animal encounter programs\u2026. See ways to experience (9) 2024. 2.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-05\u0026d=4522996157713776\u0026mkt=en-US\u0026setlang=en-US\u0026w=B123HWEJmKgI0g_BICzUUE092xyTtSjJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.1g_oKFZuxZyu0MwSWAr2YwHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cGQLTADJ7EIEL8n56X81XwHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.gpxEMgIaDnDSvyMtPJjjdgHaE6\u0026pid=Api","contentUrl":"https://wowtravel.me/wp-content/uploads/2022/06/Top-10-Things-to-Do-in-Atlanta.jpg","hostPageUrl":"https://wowtravel.me/things-to-do-in-atlanta-this-week/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.GDtNm5tAuiy9g1VcZR9skwHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/07/24-Things-to-Do-in-Downtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/things-to-do-in-downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.vjZ_V7Yjzfqyphfsj_SBjgAAAA\u0026pid=Api","contentUrl":"https://daytripqueen.com/wp-content/uploads/2021/07/Free-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://daytripqueen.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.unmT01rPD7si8w-n3kzOXgHaHa\u0026pid=Api","contentUrl":"https://cdn2.storyasset.link/5f018ed1-7c8d-49e0-9d76-582219d18631/Untitled%20design%20-1-ms-rtheoozqlb.jpg","hostPageUrl":"https://tripononline.com/web-stories/ultimate-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.JIWcXtQtpq7A0DXTjwLQiwHaLG\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/fc/4a/d2/fc4ad281802906d1cfde53017a3e4dcf.jpg","hostPageUrl":"https://www.pinterest.com/pin/744079169673803460/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Yac324FtG-dH_KY8XaUTZQHaME\u0026pid=Api","contentUrl":"https://www.frostandsun.com/wp-content/uploads/2017/11/things-to-do-in-atlanta-pin-me-1-628x1024.jpg.webp","hostPageUrl":"https://www.frostandsun.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.eKakKX-7lUL-8AAx78alPQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VOzVfWJV7_6joNkGhLP7AAHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/ce/c5/25/cec525ba57be0022de5d70d1a1f3dd88.jpg","hostPageUrl":"https://www.pinterest.com/pin/277393658288987671/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/Vl6GPUuj1mc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=Vl6GPUuj1mc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg?is-pending-load=1","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.rwusIbKd5bywaoWEipraFwAAAA\u0026pid=Api","contentUrl":"https://www.smartertravel.com/wp-content/uploads/2016/08/Atlanta-1-1024x683.jpg","hostPageUrl":"https://www.smartertravel.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TNv2qzXsSFvw43t4zu0gGgHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/3NRm4q9Qg2bdUqXvenPBVeIloV4=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.HX79HLcI16lMiECbxV7UWQHaE7\u0026pid=Api","contentUrl":"https://a.cdn-hotels.com/gdcs/production44/d1147/0769c6eb-4e78-45d3-bcca-d80c6e3ac960.jpg?impolicy=fcrop\u0026w=800\u0026h=533\u0026q=medium","hostPageUrl":"https://www.hotels.com/go/usa/things-to-do-atlanta"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on 2024-10-12", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-10-12"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Events From October 1 To The Weekend Of October 5-6 2024. \u2192 See September events. \u2192 For concerts this week, see Atlanta concerts in October 2024. \u2013 Daddy Long Legs, September 13 - October 6 2024. Musical at The Legacy Theatre, Tyrone. More details.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4907344191957257\u0026mkt=en-US\u0026setlang=en-US\u0026w=gk2PwJ4MGAK3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... 12:00 PM Fri. Days of the Dead - Friday The Courtland Grand Hotel. 30303, 165 Courtland Street NE, Atlanta, GA, US Prices from $69 Avg. price ~ $79. 2 tickets remaining! ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4898908883932841\u0026mkt=en-US\u0026setlang=en-US\u0026w=t-7sQmKPkPazxHogtJy0oBuKycEPAm_-"},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids - MSN","url":"https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","displayUrl":"https://www.msn.com/.../33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","snippet":"October 12 \u0026amp; 13; 19 \u0026amp; 20, 2024 Check out the crispiest, juiciest apples from Georgia\u0026#39;s apple capital--Ellijay. For two weekends, Ellijay hosts the Georgia Apple Festival complete with arts and ..."},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4714289709671445\u0026mkt=en-US\u0026setlang=en-US\u0026w=uP6_4hpqL9Zfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12, 2024 Lilburn City Park FREE. This beloved fall arts \u0026amp; crafts is in its 50th year! It is extremely family-friendly, with plenty of activities for the kids. You\u2019ll also find food, vendor booths, and live entertainment all day long. ... The Best Free Things to Do in Atlanta in 2024 - says: August 20, 2024 at 3:57 pm [\u2026] Castleberry ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=5012794232883316\u0026mkt=en-US\u0026setlang=en-US\u0026w=IFPPoHWFm5znjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Jazz Festival \u2013 May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15\u002B artists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4520195834871245\u0026mkt=en-US\u0026setlang=en-US\u0026w=sqN1N4yyzZUHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! DiscoverAtlanta.com homepage. Things to Do; Events; ... Multiple dates until 10/12/2024. Roberta Lea Live 10/5/24. Strand Ole Opry: Fa-BOO-lous Halloween Hits 10/5/24. The Modern x Atlas 10/5/24. Cirque ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4819881477614541\u0026mkt=en-US\u0026setlang=en-US\u0026w=cQIKbv56z1uP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"23 Things You Have to Do in Atlanta This Year, From One Musicfest to Homecoming ... By Okla Jones. Updated on July 9, 2024 at 12:40 PM. ... July 20\u201321, 2024 Atlanta Expo Center","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"27 Thrilling Things To Do In Atlanta This October - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-october/","displayUrl":"https://secretatlanta.co/things-to-do-october","snippet":"Zoo Atlanta. Every year, Zoo Atlanta hosts a trick-or-treating experience that\u2019s fun for the whole family! This year, the event will take place October 19th, 20th, 26th, and 27th, from 9 a.m. \u2013 3 p.m.! Bring the kids for trick or treating, photo ops, costumed characters, and themed activities! This is not a scary event at all!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4673096681200628\u0026mkt=en-US\u0026setlang=en-US\u0026w=_CWggicgECxNFCQACSuK0PesDCkO7vK5","thumbnailUrl":"https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"24 Exciting Things To Add To Your Atlanta Bucket List - Secret Atlanta","url":"https://secretatlanta.co/2024-atlanta-bucket-list/","displayUrl":"https://secretatlanta.co/2024-atlanta-bucket-list","snippet":"Here are 24 things to add to your Atlanta bucket list! 1. ... You don\u2019t want to miss out on Shaky Knees in 2024, Atlanta! 8. Get fancy at one of Atlanta\u2019s newly Michelin-recommended restaurants. ... This year Atlanta Pride will take place in Piedmont Park on October 12-13, 2024! For more information about Pride, check out Atlanta Pride\u2019s ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002B2024-10-12\u0026d=4710587449428523\u0026mkt=en-US\u0026setlang=en-US\u0026w=Jd7vvTAcmoleIPl9aAYvzasJhzxpOrCq","thumbnailUrl":"https://www.bing.com/th?id=OIP.Jx474uFzP4iYwa3hHq7X7AHaFn\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.EwTRMK2YnFak4c4zUFMiVAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/09/Things-to-Do-in-Downtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/things-to-do-in-downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.gpxEMgIaDnDSvyMtPJjjdgHaE6\u0026pid=Api","contentUrl":"https://wowtravel.me/wp-content/uploads/2022/06/Top-10-Things-to-Do-in-Atlanta.jpg","hostPageUrl":"https://wowtravel.me/things-to-do-in-atlanta-this-week/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cGQLTADJ7EIEL8n56X81XwHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TNv2qzXsSFvw43t4zu0gGgHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/3NRm4q9Qg2bdUqXvenPBVeIloV4=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/Vl6GPUuj1mc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=Vl6GPUuj1mc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.9mBzTpymEhPfPfnOyuyhzAAAAA\u0026pid=Api","contentUrl":"https://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta.jpg","hostPageUrl":"https://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.rwusIbKd5bywaoWEipraFwAAAA\u0026pid=Api","contentUrl":"https://www.smartertravel.com/wp-content/uploads/2016/08/Atlanta-1-1024x683.jpg","hostPageUrl":"https://www.smartertravel.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Agse3pS4C61ss3RxvVgySwHaEK\u0026pid=Api","contentUrl":"https://wirksmoving.com/wp-content/uploads/2020/04/Things-to-do-in-Atlanta-with-Kids-1536x864.png","hostPageUrl":"https://wirksmoving.com/blog/things-to-do-in-atlanta-ga-with-kids/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.vjZ_V7Yjzfqyphfsj_SBjgAAAA\u0026pid=Api","contentUrl":"https://daytripqueen.com/wp-content/uploads/2021/07/Free-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://daytripqueen.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.6_fMiPMrfeqD9K0vgkdR6QHaE8\u0026pid=Api","contentUrl":"https://www.getawaymavens.com/wp-content/uploads/2019/01/Westin-Peachtree-Atlanta.jpg","hostPageUrl":"https://www.getawaymavens.com/downtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.k7hGzw27Lx3uqX8C1eseEQHaE6\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2022/05/atlanta-1280x850.jpeg","hostPageUrl":"https://365atlantatraveler.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.PzRE6SJDtJ9H3Z9jEfK5cAHaE8\u0026pid=Api","contentUrl":"https://image.jimcdn.com/app/cms/image/transf/dimension=640x10000:format=jpg/path/s9524a0afdc5b40b0/image/i88ad01ce4abaee84/version/1483187737/8-fun-things-to-do-in-atlanta-travel-city-guide-via-just1wayticket.jpg","hostPageUrl":"https://www.justonewayticket.com/2016/06/14/8-fun-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fLae3sxHYp3BVcZ5UZwd8gHaLM\u0026pid=Api","contentUrl":"https://www.thebrokebackpacker.com/wp-content/uploads/2019/10/things-to-do-atlanta-pin-1.jpg","hostPageUrl":"https://www.thebrokebackpacker.com/things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg?is-pending-load=1","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on 2024-10-19", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-10-19"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Things to do in Atlanta, GA in October, including upcoming festivals, fall and Halloween events, Pride events, comedy, musicals, plays, races, conventions, neighborhood events, cultural events, and more. ... October 19-20 2024 Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station. More details \u2013 Johns Creek Arts ..."},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... 19 tickets remaining! Tickets; Oct. 02. 2024. 7:00 PM Wed. Dayseeker The Eastern - GA. 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia."},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898","snippet":"Things to Do in Atlanta, Georgia: See Tripadvisor\u0026#39;s 475,622 traveler reviews and photos of Atlanta tourist attractions. ... Private and Luxury (19) City Tours (16) Points of Interest \u0026amp; Landmarks (28) Half-day Tours (48) Bus Tours (60) ... Written September 27, 2024. This review is the subjective opinion of a Tripadvisor member and not of ..."},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids - MSN","url":"https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","displayUrl":"https://www.msn.com/.../33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP","snippet":"October 19 \u0026amp; 20; 26 \u0026amp; 27, 2024 Zoo Atlanta hosts Boo at the Zoo for two weekends in October. Kids come dressed in costumes for a not-so-scary Halloween festival during the day."},{"name":"Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! DiscoverAtlanta.com homepage. Things to Do; Events; ... Race for Rest 2024 10/19/24. Safe and Secure Online \u2013 Cybersecurity for Kids 10/19/24. Ser Familia\u2019s Saborea 10/19/24. Stevie Wonder Live at State ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898-Activities-zfr7917665-Atlanta_Georgia.html","snippet":"2024. 2. Atlanta Botanical Garden. 5,236. Gardens. Step into a world of magic and serenity at the Atlanta Botanical Garden, where features like the Fuqua Orchid Center treat visitors to a rare collection of high-elevation orchids never before grown\u2026. See tours. 3. World of Coca-Cola."},{"name":"THE 10 BEST Things to Do in Atlanta 2024 (with Photos) - Tripadvisor","url":"https://www.tripadvisor.com.sg/Attractions-g60898-Activities-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com.sg/Attractions-g60898-Activities-Atlanta_Georgia.html","snippet":"Georgia Aquarium. 23,862. Aquariums. Georgia Aquarium contains more than 70 amazing habitats, with thousands of marine animals such as whale sharks, manta rays, beluga whales, and more. Don\u2019t miss out on the animal encounter programs\u2026. See ways to experience (9) 2024. 2. Atlanta Botanical Garden.","thumbnailUrl":"https://www.bing.com/th?id=OIP.x3JXgLmp-OZ0ej5YnBXh6wHaGL\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 10 BEST Things to Do in Atlanta - 2024 (with Photos) - Tripadvisor","url":"https://www.tripadvisor.co.za/Attractions-g60898-Activities-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.co.za/Attractions-g60898-Activities-Atlanta_Georgia.html","snippet":"Things to Do in Atlanta, Georgia: See Tripadvisor\u0026#39;s 475 561 traveller reviews and photos of 1,433 Atlanta attractions. ... Private and Luxury (19) City Tours (16) Points of Interest \u0026amp; Landmarks (28) Half-day Tours (47) Bus Tours (58) Gardens (5) ... Written 25 September 2024. This review is the subjective opinion of a Tripadvisor member and not ..."}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on 2024-10-26", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on 2024-10-26"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Events From October 1 To The Weekend Of October 5-6 2024. \u2192 See September events. \u2192 For concerts this week, see Atlanta concerts in October 2024. \u2013 Daddy Long Legs, September 13 - October 6 2024. Musical at The Legacy Theatre, Tyrone. More details."},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month."},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 26, 2024 Oakhurst Village FREE. Local BBQ restaurants present a variety of options, including vegan and gluten-free. Deserts, frozen treats, and adult beverages are also available to purchase. Live music from noon to 4 p.m. Admission is FREE; just bring your wallet for food and drink. ... The Best Free Things to Do in Atlanta in 2024 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! DiscoverAtlanta.com homepage. Things to Do; Events; ... Every Saturday until 10/26/2024. Golden Opportunity: Botanical Illustration 10/2/24 \u2013 1/1/25. Chastian Challenge 10/2/24 \u2013 10/15/24. The Mill Town ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"THE 15 BEST Things to Do in Atlanta (2024) - Tripadvisor","url":"https://www.tripadvisor.com/Attractions-g60898-Activities-Atlanta_Georgia.html","displayUrl":"https://www.tripadvisor.com/Attractions-g60898","snippet":"Things to Do in Atlanta, Georgia: See Tripadvisor\u0026#39;s 475,622 traveler reviews and photos of Atlanta tourist attractions. ... (26) Segway Tours (4) Wine Tastings (15) Art Galleries (106) Movie Tours (12) Private Sightseeing Tours (53) Gardens (5) ... Written September 25, 2024. This review is the subjective opinion of a Tripadvisor member and not ..."},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia."},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"October 11\u201313, 2024 Various locations Atlanta Pride is back again for its annual celebration of the LGBTQ\u002B community. This year, ... October 26, 2024 Morehouse College","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Atlanta Horror Film Festival. Oct 4-6, 2024 Limelight Theater View on Map. You are cordially invited to the 18th Annual Atlanta Horror Film Festival held at Limelight Theater from October 4 - 6, 2024. Don\u0026#39;t miss this spooktacular collection of the best indie horror films from across the globe, including world premieres, short films, animation ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"33 Best Things to Do in Atlanta October 2024 with Kids","url":"https://mommypoppins.com/atlanta-kids/weekend-events/best-things-to-do-in-atlanta-in-october-halloween-weekend","displayUrl":"https://mommypoppins.com/atlanta-kids/weekend-events/best-things-to-do-in-atlanta-in...","snippet":"Head north about 1.5 hours to join the German celebration in Helen for its 53rd year. Visitors embrace all things Bavarian from food to beverages to music and dancing. Kids enjoy the festivities too with their own dance floor (and the always kid-favorite chicken dance). The 2024 Oktoberfest Parade will begin at 12:00 PM on Saturday, September 7 ..."}]}}
```

Here are some things to do in Atlanta for each of the next four weekends:

### Weekend of October 5, 2024
1. **Big Green Egg “EGGtoberfest”**: Celebrating 50 Years - October 5 ([Discover Atlanta](https://discoveratlanta.com/events/main/))
2. **Atlanta Fair**: October 4 - November 3 ([Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/))
3. **Oktoberfest in Helen**: Through Sunday, October 27 ([MSN](https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP))

### Weekend of October 12, 2024
1. **Georgia Apple Festival**: October 12 & 13; 19 & 20 ([MSN](https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP))
2. **Lilburn Daze Arts and Crafts Festival**: October 12 ([Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/))
3. **Atlanta Pride Festival**: October 12-13 ([Secret Atlanta](https://secretatlanta.co/2024-atlanta-bucket-list/))

### Weekend of October 19, 2024
1. **Boo at the Zoo**: October 19 & 20; 26 & 27 ([MSN](https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP))
2. **Race for Rest**: October 19 ([Discover Atlanta](https://discoveratlanta.com/events/main/))
3. **Ser Familia’s Saborea**: October 19 ([Discover Atlanta](https://discoveratlanta.com/events/main/))

### Weekend of October 26, 2024
1. **Oakhurst Village BBQ Festival**: October 26 ([Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/))
2. **Golden Opportunity: Botanical Illustration**: Every Saturday until October 26 ([Discover Atlanta](https://discoveratlanta.com/events/main/))
3. **Atlanta Horror Film Festival**: October 4-6 ([Experism](https://www.experism.com/atlanta/october))

For more details, you can visit the respective event links.


In [6]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output.LastMessage.DisplayAs("text/markdown");


Here are some things to do in Atlanta for each of the next four weekends:

### Weekend of October 5, 2024
1. **Big Green Egg “EGGtoberfest”**: Celebrating 50 Years - October 5 ([Discover Atlanta](https://discoveratlanta.com/events/main/))
2. **Atlanta Fair**: October 4 - November 3 ([Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/))
3. **Oktoberfest in Helen**: Through Sunday, October 27 ([MSN](https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP))

### Weekend of October 12, 2024
1. **Georgia Apple Festival**: October 12 & 13; 19 & 20 ([MSN](https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP))
2. **Lilburn Daze Arts and Crafts Festival**: October 12 ([Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/))
3. **Atlanta Pride Festival**: October 12-13 ([Secret Atlanta](https://secretatlanta.co/2024-atlanta-bucket-list/))

### Weekend of October 19, 2024
1. **Boo at the Zoo**: October 19 & 20; 26 & 27 ([MSN](https://www.msn.com/en-us/travel/tripideas/33-best-things-to-do-in-atlanta-october-2024-with-kids/ar-AA1rpsNP))
2. **Race for Rest**: October 19 ([Discover Atlanta](https://discoveratlanta.com/events/main/))
3. **Ser Familia’s Saborea**: October 19 ([Discover Atlanta](https://discoveratlanta.com/events/main/))

### Weekend of October 26, 2024
1. **Oakhurst Village BBQ Festival**: October 26 ([Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/))
2. **Golden Opportunity: Botanical Illustration**: Every Saturday until October 26 ([Discover Atlanta](https://discoveratlanta.com/events/main/))
3. **Atlanta Horror Film Festival**: October 4-6 ([Experism](https://www.experism.com/atlanta/october))

For more details, you can visit the respective event links.

## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search Test");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
